In [18]:
using Revise
using ITensors,ITensorMPS
using PastaQ
push!(LOAD_PATH,"../src/")
using RandomMeas
using ProgressBars

In [20]:
N = 8
ξ = siteinds("Qubit", N)
B = 1.
ampo = AutoMPO()
for j in 1:(N - 1)
  # Ising ZZ interactions
  ampo .+= -1, "X", j, "X", j + 1
end
for j in 1:N
  # Transverse field X
  ampo .+= -B, "Z", j
end
H = MPO(ampo,ξ)
H2 = apply(H,H)
# Density-matrix renormalization group
dmrg_iter = 5      # DMRG steps
dmrg_cutoff = 1E-10   # Cutoff
ψ0 = random_mps(ξ) # Initial state
sweeps = Sweeps(dmrg_iter)
maxdim!(sweeps, 10, 20, 30, 40, 50, 100)
cutoff!(sweeps, dmrg_cutoff)
# Run 
println("Running DMRG to get ground state of transverse field Ising model:")
E, ψ = dmrg(H, ψ0, sweeps)
println("\nGround state energy:  ", E)
println("\n---------------------------------------\n")

Running DMRG to get ground state of transverse field Ising model:
After sweep 1 energy=-9.78777885367755  maxlinkdim=4 maxerr=1.56E-16 time=0.007
After sweep 2 energy=-9.837904724978326  maxlinkdim=11 maxerr=7.04E-11 time=0.091
After sweep 3 energy=-9.837951444708578  maxlinkdim=7 maxerr=9.75E-11 time=0.007
After sweep 4 energy=-9.837951446198812  maxlinkdim=6 maxerr=7.45E-11 time=0.007
After sweep 5 energy=-9.83795144619906  maxlinkdim=6 maxerr=6.91E-11 time=0.007

Ground state energy:  -9.83795144619906

---------------------------------------



In [21]:
#Consider experimental state admixed with white noise
p = 0.05
Ide = MPO(ξ)
for i in 1:N
    Ide[i] = δ(ξ[i]',ξ[i])
end
ρ = (1-p)*outer(ψ',ψ)+p*Ide/2^N;

In [22]:
nu = 5000
NM = 1000
datat = zeros(Int8,(NM,N));
data = zeros(Int8,(nu,NM,N));

In [ ]:
Es = 0.
E2s = 0.
n = 3 #We will create three batch shadows
u = Vector{Vector{ITensor}}()
ITensors.disable_warn_order()
for r in ProgressBar(1:nu, printing_delay=20)
        push!(u,get_rotations(ξ,1))
        get_RandomMeas!(datat,ρ,u[r])
        data[r,:,:] = datat
end

In [ ]:
ρb = acquire_shadows_batch_fromdata(data,ξ,u,n)
ρm = Vector{ITensor}(undef,n)
ρm[1] = (ρb[1]+ρb[2]+ρb[3])/3/nu
ρm[2] = (multiply(ρb[1],ρb[2])+multiply(ρb[1],ρb[3])+multiply(ρb[2],ρb[3]))/3/nu
ρm[3] = multiply(multiply(ρb[1],ρb[2]),ρb[3])/nu;

In [ ]:
Es = zeros(n)
for nt in 1:n
    Es[nt] = real(trace(multiply(ρm[nt],flatten(H)),ξ)/trace(ρm[nt],ξ))
end

In [ ]:
println(Es)
using PyPlot
plot(1:n,Es,"*",ms=10)
plot(1:n,ones(n)*E,"--k")
xlabel("copies")
ylabel("energy estimation")